In [1]:
!nvidia-smi

Fri Mar  4 05:49:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro RTX 5000     Off  | 00000000:3B:00.0 Off |                  Off |
| 33%   40C    P0    50W / 230W |      0MiB / 16125MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 5000     Off  | 00000000:5E:00.0 Off |                  Off |
| 33%   41C    P0    41W / 230W |      0MiB / 16125MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  Q

# Library import

In [1]:
import os 
import re 
import copy 
import time
import json
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder


from googletrans import Translator
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torch.utils.data import Dataset, DataLoader
from torch.nn.modules.loss import _WeightedLoss
from transformers import logging, AutoTokenizer, AutoModelForSequenceClassification, AutoModel
logging.set_verbosity_error()


import warnings
warnings.filterwarnings(action='ignore')


# Hyperparameter definition
seed = 10
num_labels = 3
KorNLU_num = 50000
n_fold = 5
epochs = 100
patience = 5
max_seq_len = 103
batch_size = 128
weight_decay = 0.0001
drop_out = 0.1
label_smoothing = 0.2
learning_rate = 5e-5
max_grad_norm = 10

device = '0,1,2,3'
text = 'text'
model_name = 'klue/roberta-large'
library_name = 'AutoModelForSequenceClassification'
Model_library = eval(library_name)

model_save_folder = './RESULTS/'
model_save = '{}_{}_{}_{}_{}_{}_{}_{}_{}_fold'.format(text,
                                                    KorNLU_num,
                                                    learning_rate,
                                                    max_seq_len,
                                                    n_fold,
                                                    batch_size,
                                                    weight_decay,
                                                    drop_out,
                                                    patience)

os.environ["PYTHONHASHSEED"] = str(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]=device
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")    
print('Device: %s' % device)
if (device.type == 'cuda') or (torch.cuda.device_count() > 1):
    print('GPU activate --> Count of using GPUs: %s' % torch.cuda.device_count())

/usr/local/lib/python3.6/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cuda
GPU activate --> Count of using GPUs: 4


# Data load

* 사용 데이터: KLUE Datasets + KorNLU Datasets

* KLUE: https://github.com/KLUE-benchmark/KLUE
* KorNLU: https://github.com/kakaobrain/KorNLUDatasets

* KLUE, KorNLU 모두 특수문자를 제거하는 경우보다, KorNLU 데이터만 특수문자를 제거하는 경우가 더 성능이 높게 나와 해당 데이터만 특수문자를 제거하였습니다.

In [2]:
# KLUE data load
json_train_path = 'klue-nli-v1.1_train.json'
json_test_path = 'klue-nli-v1.1_dev.json'

with open(json_train_path, 'r', encoding="utf-8") as f:
    json_train = json.load(f)
with open(json_test_path, 'r', encoding="utf-8") as f:
    json_test = json.load(f)
    
json_train_df = pd.DataFrame(json_train)[['premise','hypothesis','gold_label']]
json_test_df = pd.DataFrame(json_test)[['premise','hypothesis','gold_label']]        
json_train_df.rename(columns = {'gold_label' : 'label'}, inplace = True)
json_test_df.rename(columns = {'gold_label' : 'label'}, inplace = True)
df1 = pd.concat([json_train_df, json_test_df]).reset_index(drop=True)

df_grp1 = df1.groupby(df1.columns.tolist()) # 전체 열 비교
df_di1 = df_grp1.groups # 딕셔너리로 만들기 
idx_T1 = [x[0] for x in df_di1.values() if len(x) == 1] # 중복X 인덱스 검토
idx_F1 = [x[0] for x in df_di1.values() if not len(x) == 1] # 중복O 인덱스 검토
df_concated1 = pd.concat([df1.loc[idx_T1,:], df1.loc[idx_F1,:]])
df_concated1 = df_concated1.dropna(how='any') # Null 값이 존재하는 행 제거
df_concated1 = df_concated1.reset_index(drop=True)

# KorNLU data load
multinli_path = 'https://github.com/kakaobrain/KorNLUDatasets/raw/master/KorNLI/multinli.train.ko.tsv'
snli_path = 'https://github.com/kakaobrain/KorNLUDatasets/raw/master/KorNLI/snli_1.0_train.ko.tsv'
xnli_dev_path = 'https://github.com/kakaobrain/KorNLUDatasets/raw/master/KorNLI/xnli.dev.ko.tsv'
xnli_test_path = 'https://github.com/kakaobrain/KorNLUDatasets/raw/master/KorNLI/xnli.test.ko.tsv'
    
multinli_data = pd.read_csv(multinli_path, sep='\t', error_bad_lines=False).sample(KorNLU_num)
snli_data = pd.read_csv(snli_path, delimiter='\t', error_bad_lines=False).sample(KorNLU_num)
xnli_dev_data = pd.read_csv(xnli_dev_path, delimiter='\t', error_bad_lines=False)
xnli_test_data  = pd.read_csv(xnli_test_path, delimiter='\t', error_bad_lines=False)

multinli_data.rename(columns = {'gold_label' : 'label', 'sentence1' : 'premise', 'sentence2' : 'hypothesis'}, inplace = True)
snli_data.rename(columns = {'gold_label' : 'label', 'sentence1' : 'premise', 'sentence2' : 'hypothesis'}, inplace = True)
xnli_test_data.rename(columns = {'gold_label' : 'label', 'sentence1' : 'premise', 'sentence2' : 'hypothesis'}, inplace = True)
xnli_dev_data.rename(columns = {'gold_label' : 'label', 'sentence1' : 'premise', 'sentence2' : 'hypothesis'}, inplace = True)    
df2 = pd.concat([multinli_data,snli_data, xnli_test_data,xnli_dev_data]).reset_index(drop=True)

df_grp2 = df2.groupby(df2.columns.tolist()) # 전체 열 비교
df_di2 = df_grp2.groups # 딕셔너리로 만들기 
idx_T2 = [x[0] for x in df_di2.values() if len(x) == 1] # 중복X 인덱스 검토
idx_F2 = [x[0] for x in df_di2.values() if not len(x) == 1] # 중복O 인덱스 검토
df_concated2 = pd.concat([df2.loc[idx_T2,:], df2.loc[idx_F2,:]])
df_concated2 = df_concated2.dropna(how='any') # Null 값이 존재하는 행 제거

df_concated2['premise'] = df_concated2['premise'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 0-9 a-z A-Z]', '', regex=True)
df_concated2['hypothesis'] = df_concated2['hypothesis'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 0-9 a-z A-Z]', '', regex=True)

df_concated2 = df_concated2[df_concated2['premise'].apply(lambda x: len(x)>=19)] # 'premise' 글자 수 18 미만인 데이터 제거 
df_concated2 = df_concated2[df_concated2['premise'].apply(lambda x: len(x)<=90)] # 'premise' 글자 수 89 초과인 데이터 제거
df_concated2 = df_concated2[df_concated2['hypothesis'].apply(lambda x: len(x)>=5)] # 'hypothesis' 글자 수 5 미만인 데이터 제거
df_concated2 = df_concated2[df_concated2['hypothesis'].apply(lambda x: len(x)<=103)] # 'hypothesis' 글자 수 103 초과인 데이터 제거
df_concated2 = df_concated2.reset_index(drop=True)


b'Skipping line 24426: expected 3 fields, saw 4\nSkipping line 156343: expected 3 fields, saw 4\nSkipping line 218766: expected 3 fields, saw 4\nSkipping line 232318: expected 3 fields, saw 4\nSkipping line 253493: expected 3 fields, saw 4\n'
b'Skipping line 265734: expected 3 fields, saw 4\nSkipping line 282588: expected 3 fields, saw 4\nSkipping line 350969: expected 3 fields, saw 4\n'


In [3]:
display(df_concated1); display(df_concated2)

,premise,hypothesis,label
0,100분간 잘껄 그래도 소닉붐땜에 2점준다,100분간 자는게 더 나았을 것 같다.,neutral
1,100분간 잘껄 그래도 소닉붐땜에 2점준다,100분간 잤다.,contradiction
2,100분간 잘껄 그래도 소닉붐땜에 2점준다,소닉붐이 정말 멋있었다.,neutral
3,101빌딩 근처에 나름 즐길거리가 많습니다.,101빌딩 근처에서 즐길거리 찾기는 어렵습니다.,contradiction
4,101빌딩 근처에 나름 즐길거리가 많습니다.,101빌딩 부근에서는 여러가지를 즐길수 있습니다.,entailment
...,...,...,...
27991,힛걸 진심 최고다 그 어떤 히어로보다 멋지다,힛걸 그 어떤 히어로보다 별로다.,contradiction
27992,힛걸 진심 최고다 그 어떤 히어로보다 멋지다,힛걸 액션 장면 진심 그 어떤 히어로보다 멋지다.,neutral
27993,힛걸 진심 최고다 그 어떤 히어로보다 멋지다,힛걸 진심 최고로 멋지다.,entailment
27994,"3층 포루는 정면 2칸, 측면 2칸의 팔작 기와지붕으로 벽면 위쪽의 판문에는 전안이...",전안은 벽면 위쪽의 판문에 설치되어 있다.,entailment


,premise,hypothesis,label
0,보조금의 75 이상이 다양한 형태의 프로세서 지원을 제공하고 있다,대부분의 사람들은 보조금이 필요하다,neutral
1,2 그레이하운드 개가 트랙을 달리고 있습니다,개가 바닥에서 잠을 잔다,contradiction
2,어쩌면 울프는 언론에 의해 잘못 이끌렸는지도 모른다 마그넷의 견해에 대한 끈질긴 요약,울프는 울프의 신념에 확고했고 마그넷의 견해에 대한 언론의 요약을 무시했다,contradiction
3,101 포와로로부터 전할 말이 있다,나는 너에게 소식을 전하러 왔다,entailment
4,101 포와로로부터 전할 말이 있다,나는 지금 너에게 아무런 소식이 없다,contradiction
...,...,...,...
85910,소화기가 트럭 후드에서 연기가 피어오르는 경마장 옆 바리케이드에 앉아 있다,트럭에 불이 붙었다,neutral
85911,식당이 있고 테이블은 모두 비어 있다,식당은 문을 닫았다,neutral
85912,하얀 개가 수영장으로 뛰어들고 있다,개가 수영장으로 뛰어들고 있다,entailment
85913,한 무리의 관중들이 남자 모래 배구 경기를 본다,남자들이 스포츠를 하고 있다,entailment


# Back Translation

* 데이터 증강에 관하여 뉴스 토픽 분류 AI 경진대회 (21.06 ~21.08) Kerry님의 ‘최종 3th : [Private 5위 - 0.83705 / Back Translation]’의 내용을 참고하여 작성하였습니다.
* 문장을 번역할 때, papago와 google translator를 사용
* 일차적으로 papago를 사용하여 번역을 진행한 후, 번역이 안되는 경우 부가적으로 google translator를 이용하여 번역을 시도
* 1-step 번역이 진행됨에 따라 공백 또는 번역이 이루어지지 않는 데이터를 nan값으로 처리 후 재번역 시도
* 2-step 번역이 진행됨에 따라 일부분만 번역된 경우 또한 재번역 시도 
* 3-step 재번역에도 일부분 번역이 안되는 단어의 경우 문장에서 분리 후 번역기로 해당 단어만 번역 시도
* 4-step 한글 -> 영어로 번역 시, 번역된 문장이 기존 문장의 길이에 대한 비율 0.5이하이면 재번역 시도

* BackTranslation을 진행할 때 papago API를 이용한 게 아니라 성능이 좋지 못하고 속도도 빠르지않아 굉장히 오래걸립니다. 오류없이 멈추지 않고 진행할 경우 대략 이틀정도 걸리네요 ㅠㅠ
Augmentation data는 제 git에 같이 올리도록 할테니, 바로 학습을 진행하실 분들은 Modeling & Prediction 목차부터 진행하시면 될 것 같습니다.

### 데이터 증강을 위한 함수 정의

In [4]:
def chrome_setting(path):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(path, options=chrome_options)
    return driver

def google_translation(dataset, raw_text_list, sk, tk):
    raw_trans_list = []
    for idx in tqdm(raw_text_list):
        translator = Translator()
        translator.raise_Exception = True
        trans = translator.translate(dataset.iloc[idx], src=sk, dest=tk)
        time.sleep(1.5)
        raw_trans_list.append(trans.text)
    
    return raw_trans_list

def papago_translation(text_data, sk, tk, driver, save_name=None, index_list=None, mode='save'):
    target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))
    trans_list = []
    
    if index_list is not None:
        final_index = index_list
    else:
        final_index = range(len(text_data))
    
    for idx in tqdm(final_index): 
        try:
            driver.get('https://papago.naver.com/?sk='+sk+'&tk='+tk+'&st='+text_data.iloc[idx])
            time.sleep(1.5)
            element=WebDriverWait(driver, 10).until(target_present)
            backtrans = element.text 

            if (backtrans=='')|(backtrans==' '):
                element=WebDriverWait(driver, 10).until(target_present)
                backtrans = element.text 
                time.sleep(0.1)
                trans_list.append(backtrans)
            else:
                trans_list.append(backtrans)
        except:
            trans_list.append('')
    
        if mode == 'save':
            if idx%100==0:
                np.save(save_name+'_{}_{}.npy'.format(0,(final_index[-1] + 1)),trans_list)
    
    driver.close()
    driver.quit()  
    os.system('killall chrome') 

    if mode == 'save':    
        np.save(save_name+'_{}_{}.npy'.format(0,(final_index[-1] + 1)),trans_list)
        print(save_name+'_{}_to_{} is translated!'.format(sk, tk))
    else:    
        return trans_list
    
    
def nan_list_retranslation(raw_array_df, train, col_name, sk, tk, path):
    
    print('nan_list re-translation.')
    # 번역이 안 된 문장이 존재하는 경우 재번역  
    raw_array_df[col_name].replace('', np.nan, inplace=True)
    raw_array_df[col_name].replace(' ', np.nan, inplace=True)
    nan_list = raw_array_df[raw_array_df[col_name].isnull()].index

    if len(nan_list) != 0:
        count = 0
        
        while len(nan_list) != 0:
            driver = chrome_setting(path)
            if count < 2:
                re_trans_list = papago_translation(train[col_name], sk, tk, driver, index_list=nan_list, mode='retry') # 원본 데이터 재번역
                
            elif count >= 2:
                re_trans_list = google_translation(train[col_name], nan_list, sk, tk) # google-translator로 재번역
            
            for idx, value in zip(nan_list, re_trans_list): 
                raw_array_df[col_name].iloc[idx] = value   
            
            driver.quit()
            raw_array_df[col_name].replace('', np.nan, inplace=True)
            raw_array_df[col_name].replace(' ', np.nan, inplace=True)

            nan_list = raw_array_df[raw_array_df[col_name].isnull()].index
            count+=1
            if count == 4:
                break
        os.system('killall chrome')
        
    return raw_array_df, nan_list


def hangul_list_retranslation(raw_array_df, train, col_name, sk, tk, path):
    
    print('hangul_list re-translation.')
    # sk -> tk 번역된 문장에 sk가 존재하는 경우 재번역 
    hangul_ind=[]
    for i in range(0,len(raw_array_df)):
        temp=re.findall('[a-zA-Z]',str(raw_array_df[col_name][i]))
        if len(temp)!=0:
            hangul_ind.append(i)

    if len(hangul_ind) != 0:
        count = 0
        
        while len(hangul_ind) != 0:
            driver = chrome_setting(path)
            
            if count < 2:
                re_trans_list = papago_translation(train[col_name], sk, tk, driver, index_list=hangul_ind, mode='retry') # 원본 데이터 재번역
                            
            elif count >= 2:
                re_trans_list = google_translation(train[col_name], hangul_ind, sk, tk) # google-translator로 재번역

            for idx, value in zip(hangul_ind, re_trans_list): 
                raw_array_df[col_name].iloc[idx] = value  
            
            driver.quit()
            hangul_ind=[]
            for i in range(0,len(raw_array_df)):
                temp=re.findall('[a-zA-Z]',str(raw_array_df[col_name][i]))
                if len(temp)!=0:
                    hangul_ind.append(i)
            count+=1
            if count == 4:
                break
            
        os.system('killall chrome')
        
    return raw_array_df, hangul_ind


def hangul_word_translation(raw_array_df, col_name, sk, tk, path):
    print('hangul_word re-translation.')
    hangul_ind=[]
    for i in range(0,len(raw_array_df)):
        temp=re.findall('[a-zA-Z]',str(raw_array_df[col_name][i]))
        if len(temp)!=0:
            hangul_ind.append(i)
                    
    if len(hangul_ind) != 0:
        count = 0
        while len(hangul_ind) != 0:
            
            if count < 1:
                for idx in tqdm(hangul_ind):
                    dictt = {}
                    words_raw = re.sub('[^A-Z a-z]', ' ', raw_array_df[col_name].iloc[idx])
                    words = words_raw.split("  ")
                    words = [x.strip() for x in words if x.strip()]
                    
                    transResult_list = []
                    for text in words: 
                        driver = chrome_setting(path)
                        driver.get('https://papago.naver.com/?sk=' + sk + '&tk=' + tk + '&hn=0&st=')
                        
                        time.sleep(1)
                        driver.find_element_by_xpath('//*[@id="sourceEditArea"]/label').send_keys(text)
                        driver.find_element_by_xpath('//*[@id="btnTranslate"]').click()
                        time.sleep(1.5)
                        transResult = driver.find_element_by_xpath('//*[@id="txtTarget"]').text
                        time.sleep(1)
                        transResult_list.append(transResult)
                        driver.quit()
                        os.system('killall chrome')
                        
                    dictt['word'] = words
                    dictt['translated_word'] = transResult_list

                    for i in range(len(dictt['word'])):
                        raw_array_df[col_name].iloc[idx] = raw_array_df[col_name].iloc[idx].replace(dictt['word'][i],dictt['translated_word'][i])
                
            elif count >= 1:
                for idx in tqdm(hangul_ind):
                    dictt = {}
                    words_raw = re.sub('[^A-Z a-z]', ' ', raw_array_df[col_name].iloc[idx])
                    words = words_raw.split("  ")
                    words = [x.strip() for x in words if x.strip()]
                    
                    transResult_list = []
                    for text in words: 
                        translator = Translator()
                        translator.raise_Exception = True
                        trans = translator.translate(text, src=sk, dest=tk)
                        time.sleep(1.5)
                        transResult_list.append(trans.text)
                        os.system('killall chrome')
                    dictt['word'] = words
                    dictt['translated_word'] = transResult_list 
                    
                    for i in range(len(dictt['word'])):
                        raw_array_df[col_name].iloc[idx] = raw_array_df[col_name].iloc[idx].replace(dictt['word'][i],dictt['translated_word'][i])
                            
            hangul_ind=[]
            for i in range(0,len(raw_array_df)):
                temp=re.findall('[a-zA-Z]',str(raw_array_df[col_name][i]))
                if len(temp)!=0:
                    hangul_ind.append(i)
                    
            count+=1
            if count >= 2:
                break
            
        os.system('killall chrome')
    return raw_array_df

def len_retranslation(raw_array_df, train, col_name, sk, tk, path):
# Attempt to re-translate a translated sentence if the translated sentence has a ratio of less than 0.5 to the length of an existing sentence
    print('len rate re-translation.')    
    retrans_ind=[]
    for i in range(0,len(raw_array_df)):
        if len(raw_array_df[col_name][i])/len(df_concated1[col_name][i])<=0.5:
            retrans_ind.append(i)
            
    retrans_ind=list(set(retrans_ind))
    count = 0
    
    while len(retrans_ind) != 0:
        driver = chrome_setting(path)
        raw_trans_list = google_translation(train[col_name], retrans_ind, sk, tk)
        
        for idx, value in zip(retrans_ind, raw_trans_list): 
            raw_array_df[col_name].iloc[idx] = value  
            
        retrans_ind=[]
        for i in range(0,len(raw_array_df)):
            if len(raw_array_df[col_name][i])/len(train[col_name][i])<=0.5:
                retrans_ind.append(i)
        
        retrans_ind=list(set(retrans_ind))   
        driver.quit()
        
        count+=1
        if count >= 2:
            break
        
        os.system('killall chrome')
    return raw_array_df

#### 한글 -> 영어 번역
* 영어로 번역된 데이터 npy로 저장 후, 번역이 제대로 진행되지 않은 부분을 위해 저장된 npy 로드하여 재번역 시도 

In [5]:
for column in ['premise', 'hypothesis']:    
    print('Col_name : '+column)

    set_setting = {'path':'/chromedriver', # your chrome driver path
                    'col_name':column,
                    'sk':'ko',
                    'tk':'en',
                    'final_save_name':'to_eng_{}'.format(column)}

    path = set_setting['path']
    col_name = set_setting['col_name']
    sk = set_setting['sk']
    tk = set_setting['tk']
    final_save_name = set_setting['final_save_name']

    driver = chrome_setting(path)
    back_translation_file = 'to_eng_{}_0_27996.npy'.format(col_name)
    
    # to_eng_premise_0_27996.npy, to_eng_hypothesis_0_27996.npy 생성 
    papago_translation(df_concated1[col_name], sk, tk, driver, save_name=final_save_name) 

    raw_array = np.load('{}'.format(back_translation_file))
    raw_array_df = pd.DataFrame(raw_array, columns=[col_name])

    nan_list = [1]
    hangul_ind = [1]
    driver.quit()
    os.system('killall chrome')
    
    # retry
    for ii in range(2):
        raw_array_df, nan_list = nan_list_retranslation(raw_array_df, df_concated1, col_name, sk, tk, path)
        raw_array_df, hangul_ind = hangul_list_retranslation(raw_array_df, df_concated1, col_name, sk, tk, path)
        raw_array_df = hangul_word_translation(raw_array_df, col_name, sk, tk, path)
        raw_array_df = len_retranslation(raw_array_df, df_concated1, col_name, sk, tk, path) 
    
    #  ko -> en 번역된 데이터 csv 파일로 저장. ex) to_eng_premise.csv, to_eng_hypothesis.csv
    raw_array_df.to_csv('{}.csv'.format(final_save_name), index=False) 
    


Col_name : premise


  7%|▋         | 1861/27996 [51:03<11:57:13,  1.65s/it]

#### 영어 -> 한글 번역
* 한글로 번역된 데이터 npy로 저장 후, 번역이 제대로 진행되지 않은 부분을 위해 저장된 npy 로드하여 재번역 시도
* 같은 의미의 문장이라도 한글 대비 영어 문장이 길다고 판단되어 길이에 대한 비율에 따른 재번역은 진행 X 

In [ ]:
for column in ['premise', 'hypothesis']:    
    print('Col_name : '+column)

    set_setting = {'path':'/home/gpuadmin/SY_LEE/chromedriver', # your chrome driver path
                    'col_name':column,
                    'sk':'en',
                    'tk':'ko',
                    'final_save_name':'to_kor_{}'.format(column)}

    path = set_setting['path']
    col_name = set_setting['col_name']
    sk = set_setting['sk']
    tk = set_setting['tk']
    final_save_name = set_setting['final_save_name']

    driver = chrome_setting(path)
    eng_data = pd.read_csv('to_eng_{}_0_27996.csv'.format(col_name))
    back_translation_file = 'to_kor_{}_0_27996.npy'.format(col_name)
    
    # to_kor_premise_0_27996.npy, to_kor_hypothesis_0_27996.npy 생성 
    papago_translation(eng_data[col_name], sk, tk, driver, save_name=final_save_name)

    raw_array = np.load('{}'.format(back_translation_file))
    raw_array_df = pd.DataFrame(raw_array, columns=[col_name])

    nan_list = [1]
    hangul_ind = [1]
    driver.quit()
    os.system('killall chrome')
    
    # retry
    for ii in range(2):
        raw_array_df, nan_list = nan_list_retranslation(raw_array_df, eng_data, col_name, sk, tk, path)
        raw_array_df, hangul_ind = hangul_list_retranslation(raw_array_df, eng_data, col_name, sk, tk, path)
        raw_array_df = hangul_word_translation(raw_array_df, col_name, sk, tk, path)
    
    #  en -> ko 번역된 데이터 csv 파일로 저장. ex) to_kor_premise.csv, to_kor_hypothesis.csv
    raw_array_df.to_csv('{}.csv'.format(final_save_name), index=False)


# Sentence BERT(SBERT) 
* 번역된 데이터와 원본 데이터와의 유사도를 구해 유사도가 0.9 이상인 데이터만을 학습에 사용

In [5]:
def pytorch_cos_sim(a: Tensor, b: Tensor):
    """
    Computes the cosine similarity cos_sim(a[i], b[j]) for all i and j.
    :return: Matrix with res[i][j]  = cos_sim(a[i], b[j])
    """
    return cos_sim(a, b)

def cos_sim(a: Tensor, b: Tensor):
    """
    Computes the cosine similarity cos_sim(a[i], b[j]) for all i and j.
    :return: Matrix with res[i][j]  = cos_sim(a[i], b[j])
    """
    if not isinstance(a, torch.Tensor):
        a = torch.tensor(a)

    if not isinstance(b, torch.Tensor):
        b = torch.tensor(b)

    if len(a.shape) == 1:
        a = a.unsqueeze(0)

    if len(b.shape) == 1:
        b = b.unsqueeze(0)

    a_norm = torch.nn.functional.normalize(a, p=2, dim=1)
    b_norm = torch.nn.functional.normalize(b, p=2, dim=1)
    return torch.mm(a_norm, b_norm.transpose(0, 1))

In [7]:
back_premise = pd.read_csv('to_kor_premise.csv')
bcak_hypothesis = pd.read_csv('to_kor_hypothesis.csv')

sim_data = copy.copy(df_concated1)
sim_data['back_premise'] = back_premise
sim_data['back_hypothesis'] = bcak_hypothesis

sim_data.replace('', np.nan, inplace=True)
sim_data.replace(' ', np.nan, inplace=True)
nan_list = [index for index, row in sim_data.iterrows() if row.isnull().any()]

hangul_ind=[]
for i in range(0,len(sim_data)):
    temp=re.findall('[a-zA-Z]',str(sim_data.drop(columns=['label']).iloc[i].values))
    if len(temp)!=0:
        hangul_ind.append(i)
        
sim_data.drop(index = hangul_ind, inplace=True)
sim_data = sim_data[sim_data['back_premise'].apply(lambda x: len(x)>=19)] # 'premise' 글자 수 18 미만인 데이터 제거 
sim_data = sim_data[sim_data['back_premise'].apply(lambda x: len(x)<=90)] # 'premise' 글자 수 89 초과인 데이터 제거
sim_data = sim_data[sim_data['back_hypothesis'].apply(lambda x: len(x)>=5)] # 'hypothesis' 글자 수 5 미만인 데이터 제거
sim_data = sim_data[sim_data['back_hypothesis'].apply(lambda x: len(x)<=103)] # 'hypothesis' 글자 수 103 초과인 데이터 제거

sim_data = sim_data.reset_index(drop=True)
sim_data[['back_premise', 'back_hypothesis']] = sim_data[['back_premise', 'back_hypothesis']].applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [8]:
display(sim_data)

,premise,hypothesis,label,back_premise,back_hypothesis
0,100분간 잘껄 그래도 소닉붐땜에 2점준다,100분간 자는게 더 나았을 것 같다.,neutral,100분 동안 더 잘할 걸 그랬어. 소닉붐 2점 드릴게요.,100분만 자는 게 더 좋았을 것 같아.
1,100분간 잘껄 그래도 소닉붐땜에 2점준다,100분간 잤다.,contradiction,100분 동안 더 잘할 걸 그랬어. 소닉붐 2점 드릴게요.,100분 잤어요.
2,100분간 잘껄 그래도 소닉붐땜에 2점준다,소닉붐이 정말 멋있었다.,neutral,100분 동안 더 잘할 걸 그랬어. 소닉붐 2점 드릴게요.,소닉 붐은 정말로 시원했다.
3,101빌딩 근처에 나름 즐길거리가 많습니다.,101빌딩 근처에서 즐길거리 찾기는 어렵습니다.,contradiction,101빌딩 근처에는 즐길 거리가 많습니다.,101빌딩 근처에서는 즐길 거리를 찾기가 힘드네요.
4,101빌딩 근처에 나름 즐길거리가 많습니다.,101빌딩 부근에서는 여러가지를 즐길수 있습니다.,entailment,101빌딩 근처에는 즐길 거리가 많습니다.,101빌딩 근처에서는 많은 것을 즐길 수 있습니다.
...,...,...,...,...,...
27010,힛걸 진심 최고다 그 어떤 히어로보다 멋지다,힛걸 그 어떤 히어로보다 별로다.,contradiction,네가 최고야. 당신은 그 어떤 영웅보다도 멋져요.,히트 소녀는 다른 영웅보다 더 나쁩니다.
27011,힛걸 진심 최고다 그 어떤 히어로보다 멋지다,힛걸 액션 장면 진심 그 어떤 히어로보다 멋지다.,neutral,네가 최고야. 당신은 그 어떤 영웅보다도 멋져요.,히트걸 액션 장면은 다른 어떤 영웅보다 정말 멋지다.
27012,힛걸 진심 최고다 그 어떤 히어로보다 멋지다,힛걸 진심 최고로 멋지다.,entailment,네가 최고야. 당신은 그 어떤 영웅보다도 멋져요.,소녀는 심각하게 가장 멋지다.
27013,"3층 포루는 정면 2칸, 측면 2칸의 팔작 기와지붕으로 벽면 위쪽의 판문에는 전안이...",전안은 벽면 위쪽의 판문에 설치되어 있다.,entailment,"3층 포루는 팔작지붕으로 정면 2칸 측면 2칸이며, 벽면 상부에 정면 조망이 설치되...",전면은 벽 위 패널에 설치되어 있습니다.


In [9]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('bespin-global/klue-sentence-roberta-base')
model = AutoModel.from_pretrained('bespin-global/klue-sentence-roberta-base')

results={}
# for col_name in ['premise']:
for col_name in ['premise', 'hypothesis']:

    # Tokenize sentences
    corpus_input = tokenizer(list(sim_data[col_name]), max_length=103, padding='max_length', truncation=True, return_tensors='pt')
    queries_input = tokenizer(list(sim_data['back_'+col_name]), max_length=103, padding='max_length', truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        corpus_output = model(**corpus_input)
        queries_output = model(**queries_input)

    # Perform pooling. In this case, mean pooling.
    corpus_embeddings = mean_pooling(corpus_output, corpus_input['attention_mask'])
    queries_embeddings = mean_pooling(queries_output, corpus_input['attention_mask'])
    results['corpus'] = corpus_embeddings
    results['queries'] = queries_embeddings
    
    cos_similarity = torch.diagonal(pytorch_cos_sim(results['corpus'], results['queries']), 0)
    sim_data[col_name+'_cos'] = cos_similarity

In [ ]:
display(sim_data[['premise','back_premise','premise_cos']])

                                                 premise  \
0                                100분간 잘껄 그래도 소닉붐땜에 2점준다   
1                                100분간 잘껄 그래도 소닉붐땜에 2점준다   
2                                100분간 잘껄 그래도 소닉붐땜에 2점준다   
3                               101빌딩 근처에 나름 즐길거리가 많습니다.   
4                               101빌딩 근처에 나름 즐길거리가 많습니다.   
...                                                  ...   
27010                           힛걸 진심 최고다 그 어떤 히어로보다 멋지다   
27011                           힛걸 진심 최고다 그 어떤 히어로보다 멋지다   
27012                           힛걸 진심 최고다 그 어떤 히어로보다 멋지다   
27013  3층 포루는 정면 2칸, 측면 2칸의 팔작 기와지붕으로 벽면 위쪽의 판문에는 전안이...   
27014                     오히려 복잡한 람블라스나 카탈루냐보다 낫다고 생각해요.   

                                            back_premise  premise_cos  
0                       100분 동안 더 잘할 걸 그랬어. 소닉붐 2점 드릴게요.     0.761380  
1                       100분 동안 더 잘할 걸 그랬어. 소닉붐 2점 드릴게요.     0.761380  
2                       100분 동안 더 잘할 걸 그랬어. 소닉붐 2점 드릴게요.     0.

In [ ]:
condition = (sim_data.premise_cos >= 0.9) & (sim_data.hypothesis_cos >= 0.9) # 조건식 작성
back_train = sim_data[condition]
back_train = back_train.reset_index(drop=True)
display(back_train[['back_premise', 'back_hypothesis', 'label']])


,back_premise,back_hypothesis,label
0,101빌딩 근처에는 즐길 거리가 많습니다.,101빌딩 근처에서는 즐길 거리를 찾기가 힘드네요.,contradiction
1,101빌딩 근처에는 즐길 거리가 많습니다.,101빌딩 주변에는 젊은이들이 즐길 수 있는 것들이 많다.,neutral
2,웬디는 10년 만에 찾는 피터에게 따뜻하게 인사하고 피터는 성공적으로 연설을 마쳤고...,웬디는 피터에게 차갑게 인사했다.,contradiction
3,웬디는 10년 만에 찾는 피터에게 따뜻하게 인사하고 피터는 성공적으로 연설을 마쳤고...,잭과 매기는 피터 배닝의 형제이다.,neutral
4,웬디는 10년 만에 찾는 피터에게 따뜻하게 인사하고 피터는 성공적으로 연설을 마쳤고...,"피터 배닝, 잭, 그리고 매기는 형제입니다.",entailment
...,...,...,...
8674,희영은 아내와 함께 세상을 떠난 뒤 아들과 단둘이 살고 있는 성현에 설렘을 느낀다.,성현은 아내와 이혼하고 혼자 살고 있습니다.,contradiction
8675,희영은 아내와 함께 세상을 떠난 뒤 아들과 단둘이 살고 있는 성현에 설렘을 느낀다.,성현이의 아내가 죽었어요.,entailment
8676,희영은 아내와 함께 세상을 떠난 뒤 아들과 단둘이 살고 있는 성현에 설렘을 느낀다.,희영 씨도 남편과 사별했습니다.,neutral
8677,"힐링푸드 전문가 양성 참여, 개발식품 활용 등의 혜택도 제공된다.",힐링 푸드 전문가들에게 혜택이 주어진다.,neutral


In [ ]:
# Augmentation data save to csv file
back_train[['back_premise', 'back_hypothesis', 'label']].to_csv('Augmented_data.csv', index=False)

# Modeling & Prediction

### Definition functions to train & test

In [10]:
class NLIDataset(Dataset):
    def __init__(self, data, tokenizer, max_seq_len=128, mode='train'):
        self.data = data
        self.max_seq_len = max_seq_len
        self.tokenizer = tokenizer
        self.mode = mode
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        
        if self.mode == 'train':
            record = self.data[index]
            encoding_result = self.tokenizer.encode_plus(record['premise'], record['hypothesis'], 
                                                    max_length=self.max_seq_len, 
                                                    pad_to_max_length=True,
                                                    truncation=True)
            return {'input_ids': np.array(encoding_result['input_ids'], dtype=int),
                    'attention_mask': np.array(encoding_result['attention_mask'], dtype=int),
                    'token_type_ids': np.array(encoding_result['token_type_ids'], dtype=int),
                    'labels': np.array(record['label_num'], dtype=int)}
            
        else:
            record = self.data.iloc[index]
            encoding_result = self.tokenizer.encode_plus(record['premise'], record['hypothesis'], 
                                                    max_length=self.max_seq_len, 
                                                    pad_to_max_length=True,
                                                    truncation=True)
            return {'input_ids': np.array(encoding_result['input_ids'], dtype=int),
                    'attention_mask': np.array(encoding_result['attention_mask'], dtype=int),
                    'token_type_ids': np.array(encoding_result['token_type_ids'], dtype=int)}


class SmoothCrossEntropyLoss(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth_one_hot(targets:torch.Tensor, n_classes:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = torch.empty(size=(targets.size(0), n_classes),
                    device=targets.device) \
                .fill_(smoothing /(n_classes-1)) \
                .scatter_(1, targets.data.unsqueeze(1), 1.-smoothing)
        return targets

    def forward(self, inputs, targets):
        targets = SmoothCrossEntropyLoss._smooth_one_hot(targets, inputs.size(-1),
            self.smoothing)
        lsm = F.log_softmax(inputs, -1)

        if self.weight is not None:
            lsm = lsm * self.weight.unsqueeze(0)

        loss = -(targets * lsm).sum(-1)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss
    

class EarlyStopping(object):
    def __init__(self, mode='min', min_delta=0, patience=10, percentage=False):
        self.mode = mode
        self.min_delta = min_delta
        self.patience = patience
        self.best = None
        self.num_bad_epochs = 0
        self.is_better = None
        self._init_is_better(mode, min_delta, percentage)

        if patience == 0:
            self.is_better = lambda a, b: True
            self.step = lambda a: False

    def step(self, metrics):
        if self.best is None:
            self.best = metrics
            return False

        if torch.isnan(metrics):
            return True

        if self.is_better(metrics, self.best):
            self.num_bad_epochs = 0
            self.best = metrics
        else:
            self.num_bad_epochs += 1

        if self.num_bad_epochs >= self.patience:
            return True

        return False

    def _init_is_better(self, mode, min_delta, percentage):
        if mode not in {'min', 'max'}:
            raise ValueError('mode ' + mode + ' is unknown!')
        if not percentage:
            if mode == 'min':
                self.is_better = lambda a, best: a < best - min_delta
            if mode == 'max':
                self.is_better = lambda a, best: a > best + min_delta
        else:
            if mode == 'min':
                self.is_better = lambda a, best: a < best - (
                            best * min_delta / 100)
            if mode == 'max':
                self.is_better = lambda a, best: a > best + (
                            best * min_delta / 100)


def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc


def predict(models,dataset,device):
    results = []
    tqdm_dataset = tqdm(enumerate(dataset), total=len(dataset))
    for batch, batch_item in tqdm_dataset:
        ids = torch.tensor(batch_item['input_ids'], dtype=torch.long, device=device)
        segment_ids = torch.tensor(batch_item['token_type_ids'], dtype=torch.long, device=device)
        atts = torch.tensor(batch_item['attention_mask'], dtype=torch.float, device=device)
        
        for fold,model in enumerate(models):
            model.eval()
            with torch.no_grad():
                if fold == 0:
                    pred = model(input_ids=ids, token_type_ids=segment_ids, attention_mask=atts)[0] 
                else:
                    pred = pred + model(input_ids=ids, token_type_ids=segment_ids, attention_mask=atts)[0] 
        pred = 0.2*pred
        pred = torch.tensor(torch.argmax(pred, axis=-1), dtype=torch.int32).cpu().numpy()
        results.extend(pred)
    return results

### Train & Validation 

* 모델은 klue_roberta_large를 사용했습니다. 5-fold를 이용하여 각 80%의 train data로 학습시킨 모델 5개로 test셋에 대해 soft voting ensemble 한 것을 최종 파일로 제출했습니다. 
* 학습 데이터: KLUE(80%) + KorNLU + Back Translation
* 검증 데이터: KLUE(20%)

* Optimizer 는 AdamW를 사용했으며, amp.GradScaler를 적용하였습니다. 또한, 모델이 과적합을 한다고 판단해 LabelSmoothing을 적용했습니다. 해당 기법 적용시 약소한 성능 향상이 있었으며,
각 fold마다 EarlyStopping을 사용하여 fold별 모델을 선택하였습니다.


* 실험은 진행하는 도중 loss는 보지 않고 acc를 위주로 보고 판단을 하였는데, acc가 높은 모델을 선택하더라도 LB에서는 오히려 더 낮게 나오는 경우가 존재했습니다.
실제로 제가 제출한 모델에서 1-fold: 10 epoch, 2-fold: 11 epoch, 3-fold: 9 epoch, 4-fold: 14 epoch, 5-fold: 14 epoch로 최종 학습한 모델을 사용한 경우 LB score는 0.89였는데, 
5-fold: 14->10 epoch으로 바꿔서 선택한 경우 LB score는 0.903으로 증가한 성능을 보였습니다.
1,2,3,4 fold에 대해서 시간관계상 진행을 못했지만, loss를 보고 판단했다면 더 좋은 성능을 보이지 않았을까 하는 생각이 있습니다.
번외로, EarlyStopping의 patience를 더 줄여서 사용하는 것도 더 좋은 성능으로 귀결되지 않을까 생각이 듭니다.



In [11]:
# Augmentation data load
AUG_data = pd.read_csv('Augmented_data.csv')
AUG_data = AUG_data.rename(columns={"back_premise": "premise", "back_hypothesis": "hypothesis"})

le = LabelEncoder()

df_concated1['label_num'] = le.fit_transform(df_concated1['label'])
df_concated2['label_num'] = le.transform(df_concated2['label'])
AUG_data['label_num'] = le.transform(AUG_data['label'])

# Dataframe convert to List
KorNLU_list = [df_concated2.iloc[i] for i in range(len(df_concated2))] 
AUG_list = [AUG_data.iloc[i] for i in range(len(AUG_data))]

In [7]:
# KFold
fold=0
folds = []
k_acc_plot, k_val_acc_plot = [], []    
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

kf = KFold(n_splits=n_fold, shuffle=True, random_state=seed)
for train_idx, valid_idx in kf.split(df_concated1):
    folds.append((train_idx, valid_idx))
    train_idx, valid_idx = folds[fold]

    Train_set = [df_concated1.iloc[i] for i in train_idx]
    Valid_set = [df_concated1.iloc[i] for i in valid_idx]
    
    # KLUE + KorNLU + Back Translation
    Train_set = Train_set + AUG_list + KorNLU_list
    
    
    # Train
    train_set = NLIDataset(data=Train_set, 
                            max_seq_len=max_seq_len,    
                            tokenizer=tokenizer)
    Train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=16, shuffle=True)
    
    # Validation 
    valid_set = NLIDataset(data=Valid_set, 
                            max_seq_len=max_seq_len, 
                            tokenizer=tokenizer)
    Valid_loader = DataLoader(valid_set, batch_size=batch_size, num_workers=16, shuffle=True)
        
    model = Model_library.from_pretrained(model_name, num_labels=num_labels).to(device) 
    model = nn.DataParallel(model).to(device)
    
    for module in model.modules():
        if isinstance(module, nn.Dropout):
            module.p = drop_out
        
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 
        'weight_decay': weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 
        'weight_decay': 0.0}
        ]

    optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=learning_rate)
    scaler = torch.cuda.amp.GradScaler()
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)
    loss_fn = SmoothCrossEntropyLoss(smoothing=label_smoothing)
    early_stopping = EarlyStopping(patience=patience, mode='max')
    
    best=0.5
    acc_plot, val_acc_plot = [], []
    for e in range(epochs):
        start=time.time()
        train_acc = 0.0
        valid_acc = 0.0
        
        model.train()
        for batch_id, batch in tqdm(enumerate(Train_loader), total=len(Train_loader)):
            optimizer.zero_grad()
            ids = torch.tensor(batch['input_ids'], dtype=torch.long, device=device)
            segment_ids = torch.tensor(batch['token_type_ids'], dtype=torch.long, device=device)
            atts = torch.tensor(batch['attention_mask'], dtype=torch.float, device=device)
            labels = torch.tensor(batch['labels'], dtype=torch.long, device=device)
            with torch.cuda.amp.autocast():    
                pred = model(input_ids=ids, token_type_ids=segment_ids, attention_mask=atts)[0] 
                pred = pred.type(torch.FloatTensor).to(device)
            loss = loss_fn(pred, labels)
            
            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            scaler.step(optimizer)
            scaler.update()
            train_acc += calc_accuracy(pred, labels)
        train_acc = train_acc/(batch_id+1)
        acc_plot.append(train_acc)
        scheduler.step()
        
        model.eval()
        for batch_id, batch in tqdm(enumerate(Valid_loader), total=len(Valid_loader)):
            with torch.no_grad():
                ids = torch.tensor(batch['input_ids'], dtype=torch.long, device=device)
                segment_ids = torch.tensor(batch['token_type_ids'], dtype=torch.long, device=device)
                atts = torch.tensor(batch['attention_mask'], dtype=torch.float, device=device)
                labels = torch.tensor(batch['labels'], dtype=torch.long, device=device)
                pred = model(input_ids=ids, token_type_ids=segment_ids, attention_mask=atts)[0]
                pred = pred.type(torch.FloatTensor).to(device)
            valid_acc += calc_accuracy(pred, labels)
        valid_acc = valid_acc/(batch_id+1)
        val_acc_plot.append(valid_acc)
        
        print_best = 0    
        if valid_acc>=best:
            difference = valid_acc - best
            best = valid_acc 
            best_idx = e+1
            model_state_dict = model.module.state_dict() if torch.cuda.device_count() > 1 else model.module.state_dict()
            best_model_wts = copy.deepcopy(model_state_dict)
            
            # load and save best model weights
            model.module.load_state_dict(best_model_wts)
            torch.save(model_state_dict, model_save_folder+model_save+str(fold)+".pt")
            print_best = '==> best model saved - %d epoch / %.5f    difference %.5f'%(best_idx, best, difference)
            
        TIME = time.time() - start
        print(f'fold : {fold+1}/{n_fold}    epoch : {e+1}/{epochs}    time : {TIME:.0f}/{TIME*(epochs-e-1):.0f} ')
        print(f'TRAIN acc : {train_acc:.5f} ')
        print(f'VALID acc : {valid_acc:.5f}    best : {best:.5f}')
        print('\n') if type(print_best)==int else print(print_best,'\n')
        
        if early_stopping.step(torch.tensor(valid_acc)):
            break
        
    fold+=1    
    k_acc_plot.append(max(acc_plot))
    k_val_acc_plot.append(max(val_acc_plot))
    
print("Train Loss: ",np.mean(k_acc_plot),", Valid Loss: ",np.mean(k_val_acc_plot))
print(model_save + ' model is saved!')
del train_set; del Train_loader; del valid_set; del Valid_loader; 
torch.cuda.empty_cache()  

100%|██████████| 44/44 [00:18<00:00,  2.42it/s]


fold : 1/5    epoch : 1/100    time : 658/65122 
TRAIN acc : 0.78067 
VALID acc : 0.92430    best : 0.92430
==> best model saved - 1 epoch / 0.92430    difference 0.42430 



100%|██████████| 44/44 [00:18<00:00,  2.42it/s]


fold : 1/5    epoch : 2/100    time : 645/63230 
TRAIN acc : 0.87331 
VALID acc : 0.92637    best : 0.92637
==> best model saved - 2 epoch / 0.92637    difference 0.00207 



100%|██████████| 44/44 [00:18<00:00,  2.43it/s]

fold : 1/5    epoch : 3/100    time : 642/62250 
TRAIN acc : 0.91737 
VALID acc : 0.91797    best : 0.92637





 51%|█████     | 466/914 [05:16<05:01,  1.49it/s]

# Inference

In [12]:
test = pd.read_csv("test_data.csv")
sub = pd.read_csv("sample_submission.csv")

label_idx = dict(zip(list(le.classes_), le.transform(list(le.classes_))))
idx_label = {value: key for key, value in label_idx.items()}

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
# Test
models = []
for fold in range(n_fold):
    model = Model_library.from_pretrained(model_name, num_labels=num_labels).to(device) 
    model = nn.DataParallel(model).to(device)
    model_dict = torch.load(model_save_folder+model_save+str(fold)+".pt")
    model.module.load_state_dict(model_dict) if torch.cuda.device_count() > 1 else model.load_state_dict(model_dict)
    models.append(model)
        
# Test 
test_set = NLIDataset(data=test, 
                        max_seq_len=max_seq_len, 
                        tokenizer=tokenizer,
                        mode='test')
Test_loader = DataLoader(test_set, batch_size=batch_size, num_workers=16, shuffle=False)

preds = predict(models,Test_loader,device)
preds = np.array([idx_label[int(val)] for val in preds])

sub["label"] = preds
sub

100%|██████████| 14/14 [00:17<00:00,  1.26s/it]


,index,label
0,0,contradiction
1,1,neutral
2,2,entailment
3,3,contradiction
4,4,contradiction
...,...,...
1661,1661,neutral
1662,1662,neutral
1663,1663,neutral
1664,1664,neutral


In [14]:
sub.to_csv("./RESULTS/{}.csv".format(model_save), index=False)
print(model_save + " is saved!")

text_50000_5e-05_103_5_128_0.0001_0.1_5_fold is saved!
